<a href="https://colab.research.google.com/github/ZacariDigital/Generative-AI-Absract-Art/blob/main/Generative_A_I_Abstract_ART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/openai/CLIP
!git clone https://github.com/CompVis/taming-transformers

In [ ]:
##Installing extra libraries
!pip install --no-deps ftfy regex tqdm
!pip install omegaconf==2.0.0 pytorch-lightning==1.0.8
!pip install torchtext
!pip install einops
!pip install torchvision

In [ ]:
!pip install --upgrade torch torchvision



In [ ]:
##importing libaries
import numpy as np
import torch, os, imageio, pdb, math
import torchvision 
import torchvision.transforms as T
import torchvision.transforms.functional as TF

import PIL 
import matplotlib.pyplot as plt 

import yaml 
from omegaconf import OmegaConf

from CLIP import clip
#import warnings
#warnings.filterwarnings('ignore')

In [ ]:
##Coding Helper functions

def show_from_tensor(tensor):
  img = tensor.clone()
  img = img.mul(255).byte()
  img = img.cpu().numpy().transponse((1,2,0))

  plt.figure(figsize=(10,7))
  plt.axis('off')
  plt.imshow(img)
  plt.show()

def norm_data(data):
  return(data.clip(-1,1)+1)/2 ##range between 0 & 1 in the result

##Parameters
learning_rate = .5
batch_size = 1
wd = .1
noise_factor = .1


total_iter=100
im_shape = [225, 400, 3] #h,w,channel
size1, size2, channels = im_shape


In [ ]:
## CLIP MODEL ##
clipmodel, _ = clip.load('ViT-B/32', jit=False)
clipmodel.eval()
print(clip.available_models())

print("Clip model visual input resolution", clipmodel.visual.input_resolution)
device=torch.device("cuda:0")
torch.cuda.empty_cache

In [ ]:
##Taming Transformer##

%cd /content/taming-transformers

!mkdir -p models/vqgan_imagenet_f16_16384/checkpoints
!mkdir -p models/vqgan_imagenet_f16_16384/configs

if len(os.listdir('models/vqgan_imagenet_f16_16384/checkpoints/')) == 0:
  !wget 'https://heibox.uni-heidelberg.de/f/867b05fc8c4841768640/?dl=1' -O 'models/vqgan_imagenet_f16_16384/checkpoints/last.ckpt'
  !wget 'https://heibox.uni-heidelberg.de/f/274fb24ed38341bfa753/?dl=1' -O 'models/vqgan_imagenet_f16_16384/configs/model.yaml'


In [ ]:
from taming.models.vqgan import VQModel

def load_config(config_path, display=False):
  config_data = OmegaConf.load(config_path)
  if display:
    print(yaml.dump(OmegaConf.to_container(config_data)))
  return config_data

def load_vqgan(config, chk_path=None):
  model = VQModel(**config.model.params)
  if chk_path is not None:
    state_dict = torch.load(chk_path, map_location="cpu")["state_dict"]
    missing, unexpected = model.load_state_dict(state_dict, strict=False)
  return model.eval()

def generator(x):
  x = taming_model.post_quant_conv(x)
  x = taming_model.decoder(x)
  return x

taming_config = load_config("./models/vqgan_imagenet_f16_16384/configs/model.yaml", display=True)
taming_model = load_vqgan(taming_config, chk_path="./models/vqgan_imagenet_f16_16384/checkpoints/last.ckpt").to(device)



In [ ]:
##Declare the values that we are going to optimize##

class Parameters(torch.nn.Module):
  def __init__(self):
    super(Parameters, self).__innit__()
    self.data = .5*torch.randn(batch_size, 256, size1//16, size2//16).cuda() #1x256x14x25 (225/16, 400/16)
    self.data = torch.nn.Parameter(torch.sin(self.data)

    def forward(self):
      return self.data

def init_params():
  params=Parameters().cuda()
  optimizer = torch.optim.Adam([{'params':[params:data], 'lr': learning_rate}], weight_decay=wd)
  return params, optimizer
